In [ ]:
# Predicting the Operons

# 1) Escherichia coli K12
# 2) Bacillus Subtilis
# 3) Halobacterium
# 4) Synechocystis


In [1]:
import gzip
import pandas as pd


def read_ptt_file(ptt_file_path):
    with gzip.open(ptt_file_path, 'rt') as file:
        lines = file.readlines()
        # Extract relevant lines, skipping header
        data_lines = lines[8:]
        # Prepare data for DataFrame
        data = []
        for line in data_lines:
            parts = line.strip().split('\t')
            location = parts[0]
            strand = parts[1]
            gene = parts[4] if len(parts) > 4 else ''
            cog = parts[7] if len(parts) > 7 else ''
            start, end = location.split('..')
            pid = parts[3]
            data.append([location, int(start), int(end), strand, gene, cog,pid])

    # Create DataFrame with the additional columns
    df = pd.DataFrame(data, columns=['Location', 'Start', 'End', 'Strand', 'Gene', 'COG','PID'])
    return df



# Read the E. coli PTT file with the updated function
ecoli_df = read_ptt_file('/content/E_coli_K12_MG1655.ptt.gz')
ecoli_df.head()


,Location,Start,End,Strand,Gene,COG,PID
0,5683..6459,5683,6459,-,yaaA,COG3022S,16128000
1,6529..7959,6529,7959,-,yaaJ,COG1115E,16128001
2,8238..9191,8238,9191,+,talB,COG0176G,16128002
3,9306..9893,9306,9893,+,mog,COG0521H,16128003
4,9928..10494,9928,10494,-,yaaH,COG1584S,16128004


In [2]:

def predict_operons(df):
    # Sort by start position
    df = df.sort_values(by=['Start'])

    # Initialize lists to hold operon data
    operons = []
    current_operon = [df.iloc[0]]  # Start with the first gene

    # Iterate over genes to group them into operons
    for i in range(1, len(df)):
        previous_gene = current_operon[-1]
        current_gene = df.iloc[i]

        # Checking if the current gene is co-directional and close to the previous gene
        if current_gene['Strand'] == previous_gene['Strand'] and (current_gene['Start'] - previous_gene['End']) < 50:
            current_operon.append(current_gene)
        else:

            operons.append(current_operon)
            current_operon = [current_gene]


    operons.append(current_operon)


    operon_summary = [{
        'Start': min(gene['Start'] for gene in operon),
        'End': max(gene['End'] for gene in operon),
        'Num_Genes': len(operon),
        'Strand': operon[0]['Strand']
    } for operon in operons]

    return pd.DataFrame(operon_summary)

# Identify operons in the E. coli dataset
ecoli_operons_df = predict_operons(ecoli_df)
ecoli_operons_df


,Start,End,Num_Genes,Strand
0,5683,6459,1,-
1,6529,7959,1,-
2,8238,9191,1,+
3,9306,9893,1,+
4,9928,10494,1,-
...,...,...,...,...
2661,4633544,4636143,3,+
2662,4636201,4637553,1,+
2663,4637613,4638329,1,-
2664,4638425,4638565,1,+


In [3]:


# Processing bsuttilis PTT file
bsubtilis_df = read_ptt_file('/content/B_subtilis_168.ptt.gz')



bsubtilis_df


,Location,Start,End,Strand,Gene,COG,PID
0,4867..6783,4867,6783,+,gyrB,COG0187L,16077074
1,6994..9459,6994,9459,+,gyrA,COG0188L,16077075
2,14847..15794,14847,15794,-,yaaC,-,16077076
3,15915..17381,15915,17381,+,guaB,COG0516F,16077077
4,17534..18865,17534,18865,+,dacA,COG1686M,16077078
...,...,...,...,...,...,...,...
4166,4211510..4212889,4211510,4212889,-,trmE,COG0486R,16081154
4167,4213200..4213826,4213200,4213826,-,jag,COG1847R,16081155
4168,4213823..4214608,4213823,4214608,-,spoIIIJ,COG0706U,16081156
4169,4214753..4215103,4214753,4215103,-,rnpA,COG0594J,16081157


In [4]:
# Processing halobacterium PTT file
halobacterium_df = read_ptt_file('/content/Halobacterium_NRC1.ptt.gz')

halobacterium_df

,Location,Start,End,Strand,Gene,COG,PID
0,7454..8641,7454,8641,-,graD5,COG1208MJ,15789346
1,8655..9860,8655,9860,-,graD2,COG1208MJ,15789347
2,9980..11413,9980,11413,+,-,COG2148M,15789348
3,11478..12614,11478,12614,-,-,COG0675L,15789349
4,12978..13187,12978,13187,+,-,COG0065E,15789350
...,...,...,...,...,...,...,...
2065,2006446..2006856,2006446,2006856,-,-,-,15791394
2066,2006920..2007438,2006920,2007438,-,-,COG1309K,15791395
2067,2007840..2009699,2007840,2009699,+,-,COG3889R,15791396
2068,2009709..2011541,2009709,2011541,-,-,-,15791397


In [5]:
# Processing synechocystis PTT file

synechocystis_df = read_ptt_file('/content/Synechocystis_PCC6803_uid159873.ptt.gz')
synechocystis_df

,Location,Start,End,Strand,Gene,COG,PID
0,5534..6622,5534,6622,-,rfbD,-,384435236
1,7229..8311,7229,8311,+,psbA2,-,384435237
2,8492..10471,8492,10471,+,speA,-,384435238
3,10622..12631,10622,12631,-,ligA,-,384435239
4,12755..13363,12755,13363,+,slr1315,-,384435240
...,...,...,...,...,...,...,...
3160,3566767..3567156,3566767,3567156,+,slr0607,-,384438396
3161,3567305..3567952,3567305,3567952,+,hisI,-,384438397
3162,3568057..3569208,3568057,3569208,+,slr0609,-,384438398
3163,3569344..3570036,3569344,3570036,+,slr0610,-,384438399


Now predicting  the operons in a Crop microbiome from Hoatzin (IMG id 2088090036)

In [6]:
import pandas as pd

def read_gff_file(gff_file):

    data = []
    with open(gff_file, 'r') as file:
        for line in file:
            if line.startswith('#'):
                continue
            parts = line.strip().split('\t')
            if len(parts) < 9:
                continue
            contig, _, _, start, end, _, strand, _, _ = parts
            data.append([contig, int(start), int(end), strand])

    data_gff = pd.DataFrame(data, columns=['Contig', 'Start', 'End', 'Strand'])
    return data_gff





In [9]:
def predict_operons(data_gff):

    operons = []
    current_operon = [data_gff.iloc[0].to_dict()]

    for _, row in data_gff.iloc[1:].iterrows():
        last_gene = current_operon[-1]
        # Checking if the current gene is co-directional and adjacent to the last gene in the operon
        if row['Strand'] == last_gene['Strand'] and (row['Start'] - last_gene['End']) < 50:
            current_operon.append(row.to_dict())
        else:

            if len(current_operon) > 1:  # Only considering groups of more than one gene as operons
                operons.append(current_operon)
            current_operon = [row.to_dict()]

    # Checking for the last operon
    if len(current_operon) > 1:
        operons.append(current_operon)


    operon_summary = [{
        'Contig': operon[0]['Contig'],
        'Start': operon[0]['Start'],
        'End': operon[-1]['End'],
        'Num_Genes': len(operon),
        'Strand': operon[0]['Strand'],


    } for operon in operons]

    return pd.DataFrame(operon_summary)




data_gff = read_gff_file('/content/2088090036.gff')


In [11]:
# Sort the dataframe by Contig and Start position for operon identification
data_sorted = data_gff.sort_values(by=['Contig', 'Start'])

# Identify operons for each contig separately and combine the results
operons_data = pd.concat([predict_operons(sub_df) for _, sub_df in data_sorted.groupby('Contig')])

operons_data



,Contig,Start,End,Num_Genes,Strand
0,HCP21_10007,3,661,2,+
0,HCP21_10010,51,666,2,-
0,HCP21_10050,1,638,2,-
0,HCP21_10055,554,1081,2,-
0,HCP21_10061,168,626,2,+
...,...,...,...,...,...
1,HCP21_singleton_3,5594,7370,4,-
0,HCP21_singleton_6,464,2772,2,+
0,HCP21_singleton_8,3,347,2,+
1,HCP21_singleton_8,491,2904,3,+
